In [ ]:
!pip install SimpleITK
!pip install optuna
!pip install monai

In [ ]:
import os
import json
from sklearn.model_selection import train_test_split
import optuna
import torch as tc
import torch.nn as nn
import json
from torchvision import models
from torchvision.models.segmentation import FCN_ResNet50_Weights, DeepLabV3_MobileNet_V3_Large_Weights
from monai.networks.nets import UNet
from monai.losses import DiceLoss

import sys
sys.path.append("/content/drive/MyDrive/Aorta_Segmentation")
from Dataset_loader import Dataset

In [ ]:
def get_files(data_path, folder):
    names = []
    folder_path = os.path.join(data_path, folder)
    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)
        for file in os.listdir(subfolder_path):
            if file.endswith(".nrrd") and not file.endswith(".seg.nrrd"):
                names.append(os.path.join(subfolder_path, file))
    return names

def split_data(names_list, pre_test_size = 0.2, val_ratio = 0.5, seed=5):
    train_names, pre_test_names = train_test_split(names_list, test_size = pre_test_size, random_state = seed)
    val_names, test_names = train_test_split(pre_test_names, test_size = val_ratio, random_state = seed)
    return train_names, val_names, test_names

Data_path = r"/content/drive/MyDrive/Aorta_Segmentation/Data"

Dongyang_data_names = get_files(Data_path, "Dongyang")
KiTS_data_names = get_files(Data_path, "KiTS")
Rider_data_names = get_files(Data_path, "Rider")

D_train_names, D_val_names, D_test_names = split_data(Dongyang_data_names)
K_train_names, K_val_names, K_test_names = split_data(KiTS_data_names)
R_train_names, R_val_names, R_test_names = split_data(Rider_data_names)

train_data_names = D_train_names + K_train_names + R_train_names
val_data_names = D_val_names + K_val_names + R_val_names
test_data_names = D_test_names + K_test_names + R_test_names

data_names_dict = {
    "train": train_data_names,
    "validation": val_data_names,
    "test": test_data_names
}

with open("/content/drive/MyDrive/Aorta_Segmentation/data_names_split.json", "w") as file:
    json.dump(data_names_dict, file, indent=2)

In [ ]:
Training_dataset = Dataset(train_data_names)
Training_dataset.Preprocess_Data()

Validation_dataset = Dataset(val_data_names)
Validation_dataset.Preprocess_Data()

In [ ]:
def Train_model(model, training_loader, validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study"):
    objective_function = DiceLoss(sigmoid=True)
    optimizer = tc.optim.Adam(model.parameters(), lr=learning_rate)
    device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
    model = model.to(device)
    if tc.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    losses = []
    val_losses = []

    best_val_loss = float("inf")
    patience = 0
    patience_limit = 3

    print(f"Training Dataset size: {len(training_loader.dataset)}") if mode == "Training" else None
    for epoch in range(num_epochs):
        print(f"Current epoch: {epoch}") if mode == "Training" else None
        epoch_loss = 0.0
        val_epoch_loss = 0.0

        model.train()
        for images, masks in training_loader:
            images, masks = images.to(device), masks.to(device)
            output = model(images.unsqueeze(1))
            if isinstance(output, dict):
                output = output["out"]
            loss = objective_function(output, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item() * images.size(0)

        model.eval()
        with tc.no_grad():
            for images, masks in validation_loader:
                images, masks = images.to(device), masks.to(device)
                output = model(images.unsqueeze(1))
                if isinstance(output, dict):
                    output = output["out"]
                val_loss = objective_function(output, masks.unsqueeze(1))
                val_epoch_loss += val_loss.item() * images.size(0)

        epoch_loss = epoch_loss / len(training_loader.dataset)
        val_epoch_loss = val_epoch_loss / len(validation_loader.dataset)
        losses.append(epoch_loss)
        val_losses.append(val_epoch_loss)
        print(f"Current epoch loss: {epoch_loss}") if mode == "Training" else None
        print(f"Current validation epoch loss: {val_epoch_loss}") if mode == "Training" else None

        if Early_stopping == True:
            if val_epoch_loss < best_val_loss:
                best_val_loss = val_epoch_loss
                patience = 0
            else:
                patience += 1
            if patience >= patience_limit:
                print(f"Early stoping triggered at epoch {epoch+1}")
                break

    return losses, val_losses, model

In [ ]:
Best_val_loss = float("inf")

def objective(trial):
    global Best_val_loss
    channels = trial.suggest_categorical("channels", [[16, 32, 64, 128], [16, 32, 64, 128, 256]])
    dropout = trial.suggest_float("dropout", 0.0, 0.2)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    num_epochs = trial .suggest_int("num_epochs", 2, 15)

    Training_loader = tc.utils.data.DataLoader(Training_dataset, batch_size=32, shuffle=True)
    Validation_loader = tc.utils.data.DataLoader(Validation_dataset, batch_size=32, shuffle=True)

    strides = (2,) * (len(channels) - 1)
    model = UNet(
        spatial_dims = 2,
        in_channels=1,
        out_channels=1,
        channels=channels,
        strides=strides,
        dropout=dropout
    )

    Final_losses, Final_val_losses, model = Train_model(model, Training_loader, Validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study")

    if Final_val_losses[-1] < Best_val_loss:
        Best_val_loss = Final_val_losses[-1]
        losses_dict = {'train_loss_lst': Final_losses, 'val_loss_lst': Final_val_losses}
        tc.save(losses_dict, '/content/drive/MyDrive/Aorta_Segmentation/Losses_values/UNet_losses.pt')
        tc.save(model.state_dict(), '/content/drive/MyDrive/Aorta_Segmentation/Models/UNet_trained.pth')

    best_epoch = Final_val_losses.index(min(Final_val_losses)) + 1
    print(f"Best epoch: {best_epoch}")
    return min(Final_val_losses)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
tc.cuda.empty_cache()
print("Best parameters:", study.best_trial.params)

[I 2025-04-16 06:53:18,252] A new study created in memory with name: no-name-ec506a12-6ac3-449b-a568-2c3719594046
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 64, 128] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 64, 128, 256] which is of type list.
  warnings.warn(message)
[I 2025-04-16 06:56:38,009] Trial 0 finished with value: 0.32724597053572957 and parameters: {'channels': [16, 32, 64, 128, 256], 'dropout': 0.05954457538304534, 'learning_rate': 0.0006718550391325637, 'num_epochs': 10}. Best is trial 0 with value: 0.32724597053572957.


Best epoch: 10


[I 2025-04-16 06:58:38,240] Trial 1 finished with value: 0.30027428993291283 and parameters: {'channels': [16, 32, 64, 128, 256], 'dropout': 0.16274768240000864, 'learning_rate': 0.0028376100987969337, 'num_epochs': 6}. Best is trial 1 with value: 0.30027428993291283.


Best epoch: 5


[I 2025-04-16 07:03:38,496] Trial 2 finished with value: 0.31409988023505225 and parameters: {'channels': [16, 32, 64, 128, 256], 'dropout': 0.005503439246068976, 'learning_rate': 0.0014823490786628252, 'num_epochs': 15}. Best is trial 1 with value: 0.30027428993291283.


...

Early stoping triggered at epoch 8
Best epoch: 5


[I 2025-04-16 07:40:42,789] Trial 18 finished with value: 0.3213723215394968 and parameters: {'channels': [16, 32, 64, 128], 'dropout': 0.056728124462157484, 'learning_rate': 0.0017276527717639011, 'num_epochs': 13}. Best is trial 13 with value: 0.2804062917405499.


Early stoping triggered at epoch 9
Best epoch: 6


[I 2025-04-16 07:43:23,441] Trial 19 finished with value: 0.28815336398521807 and parameters: {'channels': [16, 32, 64, 128, 256], 'dropout': 0.1987699033489141, 'learning_rate': 0.00969156237528058, 'num_epochs': 10}. Best is trial 13 with value: 0.2804062917405499.


Early stoping triggered at epoch 8
Best epoch: 5
Best parameters: {'channels': [16, 32, 64, 128, 256], 'dropout': 0.12643763904542435, 'learning_rate': 0.0014668839037208048, 'num_epochs': 12}


In [ ]:
Best_val_loss = float("inf")

def pytorch_objective_FCN(trial):
    global Best_val_loss
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    num_epochs = trial .suggest_int("num_epochs", 2, 15)

    Training_loader = tc.utils.data.DataLoader(Training_dataset, batch_size=64, shuffle=True)
    Validation_loader = tc.utils.data.DataLoader(Validation_dataset, batch_size=64, shuffle=True)

    model = models.segmentation.fcn_resnet50(weights=None, num_classes=1)
    model.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    Final_losses, Final_val_losses, model = Train_model(model, Training_loader, Validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study")

    if Final_val_losses[-1] < Best_val_loss:
        Best_val_loss = Final_val_losses[-1]
        losses_dict = {'train_loss_lst': Final_losses, 'val_loss_lst': Final_val_losses}
        tc.save(losses_dict, '/content/drive/MyDrive/Aorta_Segmentation/Losses_values/FCN_losses.pt')
        tc.save(model.state_dict(), '/content/drive/MyDrive/Aorta_Segmentation/Models/FCN_trained.pth')

    best_epoch = Final_val_losses.index(min(Final_val_losses)) + 1
    print(f"Best epoch: {best_epoch}")
    return min(Final_val_losses)


study = optuna.create_study(direction="minimize")
study.optimize(pytorch_objective_FCN, n_trials=5)
tc.cuda.empty_cache()
print("Best parameters deeplabv3 not pretrained:", study.best_trial.params)

[I 2025-04-16 07:43:23,468] A new study created in memory with name: no-name-e13f9321-d55e-49bc-9027-c964e6d75d96
[I 2025-04-16 09:09:31,834] Trial 0 finished with value: 0.29646253017032936 and parameters: {'learning_rate': 0.00039696059784863114, 'num_epochs': 7}. Best is trial 0 with value: 0.29646253017032936.


Best epoch: 7


[I 2025-04-16 10:10:58,182] Trial 1 finished with value: 0.3024231845084049 and parameters: {'learning_rate': 0.0006762649284194763, 'num_epochs': 5}. Best is trial 0 with value: 0.29646253017032936.


Best epoch: 4


[I 2025-04-16 12:14:06,977] Trial 2 finished with value: 0.28618906668684085 and parameters: {'learning_rate': 0.0002587757757317185, 'num_epochs': 10}. Best is trial 2 with value: 0.28618906668684085.


Best epoch: 10


[I 2025-04-16 14:14:16,473] Trial 3 finished with value: 0.3271041445739638 and parameters: {'learning_rate': 0.005400148964517831, 'num_epochs': 10}. Best is trial 2 with value: 0.28618906668684085.


Best epoch: 10


[I 2025-04-16 15:15:34,968] Trial 4 finished with value: 0.3242090319985471 and parameters: {'learning_rate': 0.0013079482845812895, 'num_epochs': 9}. Best is trial 2 with value: 0.28618906668684085.


Early stoping triggered at epoch 5
Best epoch: 2
Best parameters deeplabv3 not pretrained: {'learning_rate': 0.0002587757757317185, 'num_epochs': 10}


In [ ]:
Best_val_loss = float("inf")

def pytorch_objective_FCN_pretrained(trial):
    global Best_val_loss
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    num_epochs = trial .suggest_int("num_epochs", 2, 15)

    Training_loader = tc.utils.data.DataLoader(Training_dataset, batch_size=64, shuffle=True)
    Validation_loader = tc.utils.data.DataLoader(Validation_dataset, batch_size=64, shuffle=True)

    model = models.segmentation.fcn_resnet50(weights=FCN_ResNet50_Weights)
    model.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.classifier[4] = nn.Conv2d(512, 1, kernel_size=1)

    Final_losses, Final_val_losses, model = Train_model(model, Training_loader, Validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study")

    if Final_val_losses[-1] < Best_val_loss:
        Best_val_loss = Final_val_losses[-1]
        losses_dict = {'train_loss_lst': Final_losses, 'val_loss_lst': Final_val_losses}
        tc.save(losses_dict, '/content/drive/MyDrive/Aorta_Segmentation/Losses_values/FCN_pretrained_losses.pt')
        tc.save(model.state_dict(), '/content/drive/MyDrive/Aorta_Segmentation/Models/FCN_pretrained_trained.pth')

    best_epoch = Final_val_losses.index(min(Final_val_losses)) + 1
    print(f"Best epoch: {best_epoch}")
    return min(Final_val_losses)


study = optuna.create_study(direction="minimize")
study.optimize(pytorch_objective_FCN_pretrained, n_trials=5)
tc.cuda.empty_cache()
print("Best parameters FCN pretrained:", study.best_trial.params)

[I 2025-04-16 20:21:01,277] A new study created in memory with name: no-name-fe85989e-ff29-4ecd-8b5d-6729829e9981
[I 2025-04-16 22:28:58,651] Trial 0 finished with value: 0.5029618394111609 and parameters: {'learning_rate': 2.80792299612661e-05, 'num_epochs': 8}. Best is trial 0 with value: 0.5029618394111609.


Best epoch: 8


[I 2025-04-17 01:08:59,091] Trial 1 finished with value: 0.4195569042903768 and parameters: {'learning_rate': 3.417303211158065e-05, 'num_epochs': 10}. Best is trial 1 with value: 0.4195569042903768.


Best epoch: 10
Early stoping triggered at epoch 6


[I 2025-04-17 02:43:47,668] Trial 2 finished with value: 0.3091354641816593 and parameters: {'learning_rate': 0.0008400350504479854, 'num_epochs': 8}. Best is trial 2 with value: 0.3091354641816593.


Best epoch: 3


[I 2025-04-17 04:45:36,450] Trial 3 finished with value: 0.36074165374698697 and parameters: {'learning_rate': 0.0049394671181112555, 'num_epochs': 8}. Best is trial 2 with value: 0.3091354641816593.


Best epoch: 7
Early stoping triggered at epoch 13


[I 2025-04-17 08:12:04,070] Trial 4 finished with value: 0.2957032154599199 and parameters: {'learning_rate': 0.00022160311919661968, 'num_epochs': 14}. Best is trial 4 with value: 0.2957032154599199.


Best epoch: 10
Best parameters FCN pretrained: {'learning_rate': 0.00022160311919661968, 'num_epochs': 14}


In [ ]:
Best_val_loss = float("inf")

def objective_deeplabv3(trial):
    global Best_val_loss
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    num_epochs = trial .suggest_int("num_epochs", 2, 20)

    Training_loader = tc.utils.data.DataLoader(Training_dataset, batch_size=64, shuffle=True)
    Validation_loader = tc.utils.data.DataLoader(Validation_dataset, batch_size=64, shuffle=True)

    model = models.segmentation.deeplabv3_mobilenet_v3_large(weights=None, num_classes=1)
    model.backbone._modules["0"]._modules["0"] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)

    Final_losses, Final_val_losses, model = Train_model(model, Training_loader, Validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study")

    if Final_val_losses[-1] < Best_val_loss:
        Best_val_loss = Final_val_losses[-1]
        losses_dict = {'train_loss_lst': Final_losses, 'val_loss_lst': Final_val_losses}
        tc.save(losses_dict, '/content/drive/MyDrive/Aorta_Segmentation/Losses_values/deeplabv3_losses.pt')
        tc.save(model.state_dict(), '/content/drive/MyDrive/Aorta_Segmentation/Models/deeplabv3_trained.pth')

    best_epoch = Final_val_losses.index(min(Final_val_losses)) + 1
    print(f"Best epoch: {best_epoch}")
    return min(Final_val_losses)


study = optuna.create_study(direction="minimize")
study.optimize(objective_deeplabv3, n_trials=10)
tc.cuda.empty_cache()
print("Best parameters deeplabv3 not pretrained:", study.best_trial.params)

[I 2025-04-20 06:40:40,801] A new study created in memory with name: no-name-e48d9c22-9797-45b2-b991-ff6900c51b24
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 118MB/s] 


Early stoping triggered at epoch 6


[I 2025-04-20 06:53:02,071] Trial 0 finished with value: 0.4888653091451723 and parameters: {'learning_rate': 0.0032611910368213993, 'num_epochs': 7}. Best is trial 0 with value: 0.4888653091451723.


Best epoch: 3


[I 2025-04-20 07:09:29,499] Trial 1 finished with value: 0.47962699271150944 and parameters: {'learning_rate': 0.004706068531259832, 'num_epochs': 18}. Best is trial 1 with value: 0.47962699271150944.


...

Best epoch: 8


[I 2025-04-20 09:17:16,912] Trial 8 finished with value: 0.4947552272950061 and parameters: {'learning_rate': 0.0016536755720878017, 'num_epochs': 3}. Best is trial 6 with value: 0.4658696273523926.


Best epoch: 3


[I 2025-04-20 09:35:44,355] Trial 9 finished with value: 0.498808325277142 and parameters: {'learning_rate': 0.0055625594939345355, 'num_epochs': 9}. Best is trial 6 with value: 0.4658696273523926.


Early stoping triggered at epoch 9
Best epoch: 6
Best parameters deeplabv3 not pretrained: {'learning_rate': 0.0025929337390757047, 'num_epochs': 19}


In [ ]:
Best_val_loss = float("inf")

def objective_deeplabv3_pretrained(trial):
    global Best_val_loss
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    num_epochs = trial .suggest_int("num_epochs", 2, 20)

    Training_loader = tc.utils.data.DataLoader(Training_dataset, batch_size=64, shuffle=True)
    Validation_loader = tc.utils.data.DataLoader(Validation_dataset, batch_size=64, shuffle=True)

    model = models.segmentation.deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights)
    model.backbone._modules["0"]._modules["0"] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)
    model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

    Final_losses, Final_val_losses, model = Train_model(model, Training_loader, Validation_loader, learning_rate, num_epochs, Early_stopping = True, mode = "Study")

    if Final_val_losses[-1] < Best_val_loss:
        Best_val_loss = Final_val_losses[-1]
        losses_dict = {'train_loss_lst': Final_losses, 'val_loss_lst': Final_val_losses}
        tc.save(losses_dict, '/content/drive/MyDrive/Aorta_Segmentation/Losses_values/deeplabv3_pretrained_losses.pt')
        tc.save(model.state_dict(), '/content/drive/MyDrive/Aorta_Segmentation/Models/deeplabv3_pretrained_trained.pth')

    best_epoch = Final_val_losses.index(min(Final_val_losses)) + 1
    print(f"Best epoch: {best_epoch}")
    return min(Final_val_losses)


study = optuna.create_study(direction="minimize")
study.optimize(objective_deeplabv3_pretrained, n_trials=10)
tc.cuda.empty_cache()
print("Best parameters deeplabv3 pretrained:", study.best_trial.params)

[I 2025-04-20 19:32:24,427] A new study created in memory with name: no-name-3b0d31cf-4357-487d-80bf-d42b16dace55
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_mobilenet_v3_large-fc3c493d.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_mobilenet_v3_large-fc3c493d.pth
100%|██████████| 42.3M/42.3M [00:00<00:00, 204MB/s]
[I 2025-04-20 19:50:46,151] Trial 0 finished with value: 0.5648757538960936 and parameters: {'learning_rate': 0.0003176930713019809, 'num_epochs': 9}. Best is trial 0 with value: 0.5648757538960936.


Best epoch: 9


[I 2025-04-20 20:09:08,603] Trial 1 finished with value: 0.4747541451867823 and parameters: {'learning_rate': 0.0033518775000839275, 'num_epochs': 11}. Best is trial 1 with value: 0.4747541451867823.


Early stoping triggered at epoch 9
Best epoch: 6


[I 2025-04-20 20:29:33,695] Trial 2 finished with value: 0.48560040117438286 and parameters: {'learning_rate': 0.004348097463075958, 'num_epochs': 10}. Best is trial 1 with value: 0.4747541451867823.


...

Best epoch: 14


[I 2025-04-20 23:01:29,512] Trial 8 finished with value: 0.5507389643590909 and parameters: {'learning_rate': 0.00027304802088559145, 'num_epochs': 11}. Best is trial 1 with value: 0.4747541451867823.


Best epoch: 11


[I 2025-04-20 23:24:09,101] Trial 9 finished with value: 0.5028139884915638 and parameters: {'learning_rate': 0.0010745977628906478, 'num_epochs': 14}. Best is trial 1 with value: 0.4747541451867823.


Early stoping triggered at epoch 11
Best epoch: 8
Best parameters deeplabv3 pretrained: {'learning_rate': 0.0033518775000839275, 'num_epochs': 11}
